In [1]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import re
import urllib.request as ureq
import pymysql

In [2]:
def save_img(src, title):
    pattern = r'[가-힣a-zA-z0-9]+'
    result = re.findall(pattern, title)
    str = " ".join(result)

    path = f"../data/aladin_images/{str}.jpg"
    
    ureq.urlretrieve(src, path)
    print(path, "저장됨.")

In [ ]:
# 튜플 리스트 만들기

base = "https://www.aladin.co.kr/shop/common/wbest.aspx?BestType=Bestseller&BranchType=1&CID=0&page="

books = []

for i in range(3):
    url = base + str(i+1)
    res = requests.get(url)
    soup = bs(res.text, "html.parser")

    item_list = soup.select("div.ss_book_box")

    for item in item_list:
        title = item.select_one("a.bo3").text
        if item.select_one("li:nth-child(3) > a") != None:
            author = item.select_one("li:nth-child(3) > a").text
        else:
            author = item.select_one("li:nth-child(2) > a").text
        price = int(item.select_one("li:nth-child(4) > span").text.replace(",", ""))
        if item.select_one("img.front_cover") != None:
            img = item.select_one("img.front_cover")["src"]
        else: img = None
        
        books.append((title, author, price, img))
        if img != None: save_img(img, title)


In [4]:
# 데이터프레임 생성 및 csv 저장

df = pd.DataFrame(books, columns=["제목", "저자", "가격", "이미지소스"])
df.to_csv("../data/알라딘 실습.csv", encoding="utf-8")

df

,제목,저자,가격,이미지소스
0,소년이 온다,한강,15000,https://image.aladin.co.kr/product/4086/97/cov...
1,스토너,존 윌리엄스,16800,https://image.aladin.co.kr/product/5100/19/cov...
2,그릿 Grit,김주환,18800,https://image.aladin.co.kr/product/35853/58/co...
3,봄밤의 모든 것,백수린,17000,https://image.aladin.co.kr/product/35918/62/co...
4,양심,최재천,18000,https://image.aladin.co.kr/product/35526/93/co...
...,...,...,...,...
145,사회복지학개론 (권중돈 외),권중돈,27000,https://image.aladin.co.kr/product/33419/87/co...
146,수학 진짜 잘하는 법을 알려줄게요.,주단,22000,https://image.aladin.co.kr/product/35839/58/co...
147,어느 날 문이 사라졌다,김은영,13500,https://image.aladin.co.kr/product/35706/26/co...
148,일의 감각,조수용,22000,https://image.aladin.co.kr/product/35225/12/co...


In [ ]:
# 데이터베이스 삽입

conn = pymysql.connect(
    host="localhost",
    user="jhm",
    password="1234",
    database="pydb",
    charset="utf8"
)
c = conn.cursor()
sql = "INSERT INTO aladin_books(title, author, price, img_src) VALUES(%s, %s, %s, %s)"
c.executemany(sql, books)

conn.commit()
conn.close()
